# Process Data as per Daniel Holden's papers:

* Phase-Functioned Neural Networks for Character Control ("PFNN") (SIGGRAPH, 2017)
* A Deep Learning Framework for Character Motion Synthesis and Editing ("DLFCMS") (SIGGRAPH, 2016)

But adapted for our purposes as described below. I can't claim full understanding of every transformation, most notably the ones which use some undocumented Quaternion-based transformations. But I believe I understand it well enough that the changes are sensible. 

The only major question I have left is whether it is ok to leave out the `velocity` of each of the $21\times3$ joints. As far as I'm concerned this is just a first difference of the (transformed/rotated) joint positions, which is a derived quantity that can be derived by the model. I guess the models corresponding to the above papers weren't sequential and so couldn't do this? Anyway, it is omitted for now, and I'm just hoping for the best!

Note that the headers are in HTML in the below and don't render on Github, perhaps for security reasons.

In [1]:
import os
import sys
import numpy as np
import scipy.io as io
import scipy.ndimage.filters as filters
import pandas as pd

In [1049]:
import BVH
import Animation
from Quaternions import Quaternions
from Pivots import Pivots

<div style="background-color:#40F040;">
    <br>
    <center><b><span style="font-size:36px"> Data pre-processing Script from DH. </span></b></center><br>
        <br>
    <center><b><span style="font-size:16px"> Adapted in the following ways: </span></b></center><br>
        <ul>
         <li> Combining trajectory calc from PFNN with script from DLFCMS.
         <li> Use "local space" (Lagrangian) view from PFNN instead of quasi-Eulerian from DLFCMS.
         <li> Use cur joint positions/rotations as input too (sort of res-net idea).
         <li> Change some other attributes saved in x and y.
         <li> Save entire series rather than windows. (Done later)
        </ul>
    <br>
</div>

In [1731]:
def process_file(filename, window=240, window_step=120, export_trajectory=False):
    
    anim, names, frametime = BVH.load(filename)
    
    """ Subsample to 60 fps """
    anim = anim[::2]
    
    """ Do FK """
    global_xforms = Animation.transforms_global(anim)  # intermediate
    global_positions = global_xforms[:,:,:3,3] / global_xforms[:,:,3:,3]
    global_rotations = Quaternions.from_transforms(global_xforms)
    

    """ Remove Uneeded Joints """ #>># done post-hoc in PFNN
    used_joints = np.array([
         0,
         2,  3,  4,  5,
         7,  8,  9, 10,
        12, 13, 15, 16,
        18, 19, 20, 22,
        25, 26, 27, 29])
         
    positions = global_positions[:,used_joints]
    global_rotations = global_rotations[:,used_joints]
    # ________________________________________________________

    """ Put on Floor """
    positions[:,:,1] -= positions[:,:,1].min()
    
    """ Get Foot Contacts """
    if True:
        velfactor, heightfactor = np.array([0.05,0.05]), np.array([3.0, 2.0])
        
        fid_l, fid_r = np.array([3,4]), np.array([7,8])
        feet_l_x = (positions[1:,fid_l,0] - positions[:-1,fid_l,0])**2
        feet_l_y = (positions[1:,fid_l,1] - positions[:-1,fid_l,1])**2
        feet_l_z = (positions[1:,fid_l,2] - positions[:-1,fid_l,2])**2
        feet_l_h = positions[:-1,fid_l,1]
        feet_l = (((feet_l_x + feet_l_y + feet_l_z) < velfactor) & (feet_l_h < heightfactor)).astype(np.float)
        
        feet_r_x = (positions[1:,fid_r,0] - positions[:-1,fid_r,0])**2
        feet_r_y = (positions[1:,fid_r,1] - positions[:-1,fid_r,1])**2
        feet_r_z = (positions[1:,fid_r,2] - positions[:-1,fid_r,2])**2
        feet_r_h = positions[:-1,fid_r,1]
        feet_r = (((feet_r_x + feet_r_y + feet_r_z) < velfactor) & (feet_r_h < heightfactor)).astype(np.float)
    
    """ Get Root Velocity """
    velocity = (positions[1:,0:1] - positions[:-1,0:1]).copy()
    
    """ Remove Translation """
    positions[:,:,0] = positions[:,:,0] - positions[:,0:1,0]
    positions[:,:,2] = positions[:,:,2] - positions[:,0:1,2]
    
    """ Get Forward Direction """
    sdr_l, sdr_r, hip_l, hip_r = 13, 17, 1, 5
    across1 = positions[:,hip_l] - positions[:,hip_r]
    across0 = positions[:,sdr_l] - positions[:,sdr_r]
    across = across0 + across1
    across = across / np.sqrt((across**2).sum(axis=-1))[...,np.newaxis]
    
    direction_filterwidth = 20
    forward = np.cross(across, np.array([[0,1,0]]))
    forward = filters.gaussian_filter1d(forward, direction_filterwidth, axis=0, mode='nearest')    
    forward = forward / np.sqrt((forward**2).sum(axis=-1))[...,np.newaxis]
    
    """ Get Root Rotation """
    target = np.array([[0,0,1]]).repeat(len(forward), axis=0)
    root_rotation = Quaternions.between(forward, target)[:,np.newaxis]   # rotation needed fwd->z? 
    rvelocity = (root_rotation[1:] * -root_rotation[:-1]).to_pivots()
    
    """ Local Space """  # NEW: define position of joints relative to 
    local_positions = positions.copy()
    local_positions[:,:,0] = local_positions[:,:,0] - local_positions[:,0:1,0]  # x rel to root x
    local_positions[:,:,2] = local_positions[:,:,2] - local_positions[:,0:1,2]  # z rel to root z
    
    local_positions = root_rotation[:-1] * local_positions[:-1]   # remove Y rotation from pos
    local_velocities = local_positions[1:] - local_positions[:-1]
    local_rotations = abs((root_rotation[:-1] * global_rotations[:-1])).log()

    root_rvelocity = Pivots.from_quaternions(root_rotation[1:] * -root_rotation[:-1]).ps
    global_velocities = root_rotation[:-1] * velocity              # remove Y rotation from vel
    
    # ================================================================
    #                Construct Model Matrices
    # ================================================================
    Xc, Yc = [], []
    window = 60
    ixs = range(window, len(anim)-window-1, 1)
    
    """ Construct X Vectors (Inputs) """
    for i in ixs:
        cur_rot = root_rotation[i:i+1,0]
        cur_pos = global_positions[i:i+1,0]  # root joint
        rootposs = cur_rot * (global_positions[i-window:i+window:10,0] - cur_pos)
        rootdirs = cur_rot * forward[i-window:i+window:10]
        
        Xc.append(np.hstack([
                rootposs[:,0].ravel(), rootposs[:,2].ravel(), # Trajectory Pos (12x2)
                rootdirs[:,0].ravel(), rootdirs[:,2].ravel(), # Trajectory Dir (12x2)
                local_positions[i-1].ravel(),  # Cur Joint Pos (note -1 b.c. first el omitted)
                local_velocities[i-1].ravel(), # Cur Joint Vel (note -1 b.c. first el omitted)
                local_rotations[i-1].ravel()   # Cur Joint Rot (note -1 b.c. first el omitted)
                ]))

    """ Construct Y Vectors (Outputs) """
    for i in ixs:
        next_rot = root_rotation[i+1:i+2,0]
        next_pos = global_positions[i+1:i+2,0]  # root joint
        rootposs_next = next_rot * (global_positions[i+1:i+window+1:10,0] - next_pos)
        rootdirs_next = next_rot * forward[i+1:i+window+1:10]
        
        Yc.append(np.hstack([
                root_rvelocity[i].ravel(),     # rotational vel at root
                global_velocities[i,:,0].ravel(),  # x- vel at root
                global_velocities[i,:,2].ravel(),  # z- vel at root
                np.concatenate([feet_l[i], feet_r[i]], axis=-1), # feet contacts
                rootposs_next[:,0].ravel(), rootposs_next[:,2].ravel(), # Next Trajectory Pos
                rootdirs_next[:,0].ravel(), rootdirs_next[:,2].ravel(), # Next Trajectory Dir
                local_positions[i].ravel(),  # Next Joint Pos
                local_velocities[i].ravel(), # Next Joint Vel
                local_rotations[i].ravel()   # Next Joint Rot
                ]))
    
    return Xc, Yc

<div style="background-color:#40F040;">
    <br>
    <center><b><span style="font-size:36px">Loop over all files, adding an additional dimension at start of each </span></b></center><br><br>
</div>

In [1312]:
path = ''
database = 'cmu'

cmu_loco = pd.read_csv("../cmu/cmu_locomotion_lkp.txt", delimiter="\t", header=None)[0]

files = [os.path.join(path+database,f + ".bvh") for f in cmu_loco]
files = list(filter(lambda x: os.path.isfile(x) and x != 'rest.bvh', files))
files_empty = np.zeros(len(files), dtype='bool')

In [1762]:
Xs, Ys = [], []
for fnum, f in enumerate(files):
    X, Y = process_file(f)
    
    if len(X) > 0:
        # Convert to Float32 (reduce memory footprint)
        X = [X[i].astype('float32') for i in range(len(X))]
        Y = [Y[i].astype('float32') for i in range(len(Y))]

        # Remove velocities, and predictions of trajectory at (t+1) -- assume we always have inputs:
        X = [X[i][[*list(range(0,24)), *list(range(24,24+63)), *list(range(24+63*2,24+63*3))]] 
                 for i in range(len(X))]
        #         root rvel, root xvel, root zvel, foots(4), all joint pos(63), all joint rot(63)
        Y = [Y[i][[0,1,2, 3,4,5,6, *list(range(31,31+63)), *list(range(31+63*2,31+63*3))]] 
                 for i in range(len(Y))]
        Xs.append(np.array(X))
        Ys.append(np.array(Y))
    else:
        files_empty[fnum] = True
        
    if (fnum % 10) == 9: print(f"Processed {fnum+1} files...")

Processed 10 files...
Processed 20 files...
Processed 30 files...
Processed 40 files...
Processed 50 files...
Processed 60 files...
Processed 70 files...
Processed 80 files...
Processed 90 files...
Processed 100 files...
Processed 110 files...
Processed 120 files...
Processed 130 files...
Processed 140 files...
Processed 150 files...
Processed 160 files...
Processed 170 files...
Processed 180 files...
Processed 190 files...
Processed 200 files...


<div style="background-color:#40F040;">
    <br>
    <center><b><span style="font-size:36px">Extract Windows </span></b></center><br><br>
</div>

In [1785]:
""" Slide over windows """
def windowfy(X, Y, window=240, window_step=120):
    n = X.shape[0]
    assert X.shape[0] == Y.shape[0]
    
    windowsY = []
    windowsX = []
    l = []
    if len(Y) >= 50:
        for j in range(0, max(len(Y)-window_step-10,1), window_step):
            windowsY.append(Y[j:j+window])
            windowsX.append(X[j:j+window])
            l.append(X[j:j+window].shape[0])
    return windowsX, windowsY, l

In [1786]:
Xfull, Yfull = Xs, Ys
Xs, Ys, ls = [], [], []
for i in range(len(Xfull)):
    x, y, l = windowfy(Xfull[i], Yfull[i])
    Xs.extend(x)
    Ys.extend(y)
    ls.extend(l)

**Note that we still cannot collapse to tensor because of differing lengths**

<div style="background-color:#40F040;">
    <br>
    <center><b><span style="font-size:36px">Normalize X, Y </span></b></center><br><br>
</div>

In [1826]:
""" Calculate mean/s.d. by constructing temporary tensor of all Xs/Ys"""

tmpX = np.ones((len(Xs), 240, Xs[0].shape[1]), dtype='float32') * np.nan
for i, x in enumerate(Xs):
    tmpX[i,:x.shape[0], :x.shape[1]] = x

Xmean = np.nanmean(tmpX.reshape(-1, tmpX.shape[2]), axis=0)
Xstd  = np.nanstd(tmpX.reshape(-1, tmpX.shape[2]), axis=0)
tmpX = []

tmpY = np.ones((len(Ys), 240, Ys[0].shape[1]), dtype='float32') * np.nan
for i, y in enumerate(Ys):
    tmpY[i,:y.shape[0], :y.shape[1]] = y

Ymean = np.nanmean(tmpY.reshape(-1, tmpY.shape[2]), axis=0)
Ystd  = np.nanstd(tmpY.reshape(-1, tmpY.shape[2]), axis=0)
tmpY = []

In [1831]:
j = 21
w = 6  #((60*2)//10)
doSave = False

""" pool standard deviation over relevant dimensions of X"""
Xstd[w*0:w* 1] = Xstd[w*0:w* 1].mean() # Trajectory Past Positions
Xstd[w*1:w* 2] = Xstd[w*1:w* 2].mean() # Trajectory Future Positions
Xstd[w*2:w* 3] = Xstd[w*2:w* 3].mean() # Trajectory Past Directions
Xstd[w*3:w* 4] = Xstd[w*3:w* 4].mean() # Trajectory Future Directions

""" normalise across all joint position / rotation stds. """
# I find this slightly sketchy, but I guess it has the effect of ensuring
# that the model does care more about highly variable things, and less about
# joints with little variation, which is prob. good.
""" also reduce influence of joints by dividing all through by 10"""
# since the std is the denominator, we do this by inflating the std by *10.
def ix_sec(x): return w*4+j*3*x
Xstd[ix_sec(0):ix_sec(1)] = Xstd[ix_sec(0):ix_sec(1)].mean() / (0.1) # Pos
Xstd[ix_sec(1):ix_sec(2)] = Xstd[ix_sec(0):ix_sec(1)].mean() / (0.1) # Vel

""" pool standard deviation over relevant dimensions of Y"""
Ystd[3:7] = Ystd[3:7].mean() # Foot Contacts

def ix_sec(y): return 7+j*3*y
Ystd[ix_sec(0):ix_sec(1)] = Ystd[ix_sec(0):ix_sec(1)].mean() # Pos
Ystd[ix_sec(1):ix_sec(2)] = Ystd[ix_sec(1):ix_sec(2)].mean() # Rotation

""" Save Mean / Std """
if doSave:
    path = "../../../mocap-mtds/model_data/"
    np.savez_compressed(os.path.join(path, "Xmean.npz"), Xmean=Xmean)
    np.savez_compressed(os.path.join(path, "Ymean.npz"), Ymean=Ymean)
    np.savez_compressed(os.path.join(path, "Xstd.npz"), Xstd=Xstd)
    np.savez_compressed(os.path.join(path, "Ystd.npz"), Ystd=Ystd)

In [ ]:
""" Normalize Data """
for i, x in enumerate(Xs):
    Xs[i] = (x - Xmean) / Xstd
for i, y in enumerate(Ys):
    Ys[i] = (y - Ymean) / Ystd

<div style="background-color:#40F040;">
    <br>
    <center><b><span style="font-size:36px">We can reconstruct absolute position using the below</span></b></center><br><br>
</div>

In [1869]:
loadXmu = np.load(os.path.join(path, "Xmean.npz"))["Xmean"]
loadYmu = np.load(os.path.join(path, "Ymean.npz"))["Ymean"]
loadXstd = np.load(os.path.join(path, "Xstd.npz"))["Xstd"]
loadYstd = np.load(os.path.join(path, "Ystd.npz"))["Ystd"]

In [1874]:
def reconstruct_positions(Y, old_version=False, denormalize=True):
    Y = Y.copy()
    if denormalize:
        Y = Y * loadYstd + loadYmu
    if not old_version:
        root_r, root_x, root_z, joints = Y[:,0], Y[:,1], Y[:,2], Y[:,7:(63+7)]
    else:
        joints, root_x, root_z, root_r = Y[:,:-7], Y[:,-7], Y[:,-6], Y[:,-5]
    joints = joints.reshape((len(joints), -1, 3))

    rotation = Quaternions.id(1)
    offsets = []
    translation = np.array([[0,0,0]])

    for i in range(len(joints)):
        joints[i,:,:] = rotation * joints[i]
        joints[i,:,0] = joints[i,:,0] + translation[0,0]
        joints[i,:,2] = joints[i,:,2] + translation[0,2]
        rotation = Quaternions.from_angle_axis(-root_r[i], np.array([0,1,0])) * rotation
        offsets.append(rotation * np.array([0,0,1]))
        translation = translation + rotation * np.array([root_x[i], 0, root_z[i]])
    
    return joints

In [1881]:
Yhat = reconstruct_positions(Ys[5])

In [ ]:
animation_plot(Ys[400], fname="test2")

<div style="background-color:#40F040;">
    <br>
    <center><b><span style="font-size:36px">save out for Julia </span></b></center><br><br>
</div>

In [1891]:
np.savez_compressed(os.path.join(path, "X_input_data.npz"), Xs=Xs)
np.savez_compressed(os.path.join(path, "Y_output_data.npz"), Ys=Ys)
np.savez_compressed(os.path.join(path, "lengthsXY.npz"), ls=ls)

<div style="background-color:#40F040;">
    <br>
    <center><b><span style="font-size:36px">Animation Util </span></b></center><br><br>
</div>

## Visualisation / Animation

In [50]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
import matplotlib.colors as colors
from matplotlib.animation import ArtistAnimation
import matplotlib.patheffects as pe

In [1885]:
def animation_plot(data, fname="animation", interval=33.33, old_version=False, denormalize=True):
    
    # _____ INPUT CHECKING ______________
    if isinstance(data, np.ndarray) and data.ndim == 2:
        data = [data]
    elif isinstance(data, list):
        if isinstance(data[0], np.ndarray) and data[0].ndim == 2:
            assert len(data) <= 4, f"data list is length {len(data)} > 4: too many elements."
        else:
            raise Exception(f"Not sure what to do with list of type {type(data[0])}. Need list of 2D arrays.")
    else:
        raise Exception(f"Not sure what to do with type {type(data)}. Need (list of) 2D arrays.")
    # ___________________________________
    
    data = [reconstruct_positions(x, old_version=old_version, denormalize=denormalize) for x in data]
#     footsteps = [x[:,3:7] for x in data]
    
#     plt.scatter(data[0][ii,:,0], data[0][ii,:,1]); plt.gca().set_aspect("equal")
    
    """ Set up figure """
    scale = 1.25*((len(data))/2)
    fig = plt.figure(figsize=(12,8))
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlim3d(-scale*30, scale*30)
    ax.set_zlim3d( 0, scale*60)
    ax.set_ylim3d(-scale*30, scale*30)
    ax.set_aspect('equal')
    ax.set_xticks([], []); ax.set_yticks([], []); ax.set_zticks([], [])
    
    acolors = list(sorted(colors.cnames.keys()))[::-1]
    
    """ Initialise line objects in 3D Plot (at origin) """
    lines = []
    for yi, Y in enumerate(data):
        lines.append([plt.plot([0,0], [0,0], [0,0], color=acolors[yi], 
            lw=2, path_effects=[pe.Stroke(linewidth=3, foreground='black'), pe.Normal()])[0] 
                      for _ in range(Y.shape[1])])
    plt.tight_layout()
    
    """ DEFINE FUNC: Update lines according to animation data """
    parents = np.array([-1,0,1,2,3,0,5,6,7,0,9,10,11,11,13,14,15,11,17,18,19])
    
    def animate(t):
        changed = []        
        for ii in range(len(data)):
            offset = 25*(ii-((len(data))/2))
            for j in range(len(parents)):
                if parents[j] != -1:
                    lines[ii][j].set_data(
                        [ data[ii][t,j,0]+offset, data[ii][t,parents[j],0]+offset],
                        [-data[ii][t,j,2],       -data[ii][t,parents[j],2]])
                    lines[ii][j].set_3d_properties(
                        [ data[ii][t,j,1],        data[ii][t,parents[j],1]])
            changed += lines
            
        return changed
    
    """ Perform and save animation via Matplotlib """
    ani = animation.FuncAnimation(fig, 
        animate, np.arange(len(data[0])), interval=interval)
    
    ani.save(f'{fname}.mp4', writer=writer)
    plt.show()

In [ ]:
animation_plot(Ys[200], fname="test")

-------------

<div style="background-color:#802050;">
    <br>
    <center><b><span style="font-size:36px;color:white">OLD   VERSION </span></b></center><br><br>
</div>

In [1670]:
def process_file2(filename, window=240, window_step=120, export_trajectory=False):
    
    anim, names, frametime = BVH.load(filename)
    
    """ Subsample to 60 fps """
    anim = anim[::2]
    
    """ Do FK """
    global_positions = Animation.positions_global(anim)
    
    """ Remove Uneeded Joints """
    used_joints = np.array([
         0,
         2,  3,  4,  5,
         7,  8,  9, 10,
        12, 13, 15, 16,
        18, 19, 20, 22,
        25, 26, 27, 29])
         
    positions = global_positions[:,used_joints]
    
    """ Put on Floor """
    positions[:,:,1] -= positions[:,:,1].min()
    
    """ Get Foot Contacts """
    velfactor, heightfactor = np.array([0.05,0.05]), np.array([3.0, 2.0])
    
    fid_l, fid_r = np.array([3,4]), np.array([7,8])
    feet_l_x = (positions[1:,fid_l,0] - positions[:-1,fid_l,0])**2
    feet_l_y = (positions[1:,fid_l,1] - positions[:-1,fid_l,1])**2
    feet_l_z = (positions[1:,fid_l,2] - positions[:-1,fid_l,2])**2
    feet_l_h = positions[:-1,fid_l,1]
    feet_l = (((feet_l_x + feet_l_y + feet_l_z) < velfactor) & (feet_l_h < heightfactor)).astype(np.float)
    
    feet_r_x = (positions[1:,fid_r,0] - positions[:-1,fid_r,0])**2
    feet_r_y = (positions[1:,fid_r,1] - positions[:-1,fid_r,1])**2
    feet_r_z = (positions[1:,fid_r,2] - positions[:-1,fid_r,2])**2
    feet_r_h = positions[:-1,fid_r,1]
    feet_r = (((feet_r_x + feet_r_y + feet_r_z) < velfactor) & (feet_r_h < heightfactor)).astype(np.float)
    
    """ Get Root Velocity """
    velocity = (positions[1:,0:1] - positions[:-1,0:1]).copy()
    
    """ Remove Translation """
    positions[:,:,0] = positions[:,:,0] - positions[:,0:1,0]
    positions[:,:,2] = positions[:,:,2] - positions[:,0:1,2]
    
    """ Get Forward Direction """
    sdr_l, sdr_r, hip_l, hip_r = 13, 17, 1, 5
    across1 = positions[:,hip_l] - positions[:,hip_r]
    across0 = positions[:,sdr_l] - positions[:,sdr_r]
    across = across0 + across1
    across = across / np.sqrt((across**2).sum(axis=-1))[...,np.newaxis]
    
    direction_filterwidth = 20
    forward = np.cross(across, np.array([[0,1,0]]))
    forward = filters.gaussian_filter1d(forward, direction_filterwidth, axis=0, mode='nearest')    
    forward = forward / np.sqrt((forward**2).sum(axis=-1))[...,np.newaxis]

    """ Remove Y Rotation """
    target = np.array([[0,0,1]]).repeat(len(forward), axis=0)
    rotation = Quaternions.between(forward, target)[:,np.newaxis]    
    positions = rotation * positions
    
    """ Get Root Rotation """
    velocity = rotation[1:] * velocity
    rvelocity = (rotation[1:] * -rotation[:-1]).to_pivots()
    
    """ Add Velocity, RVelocity to vector """
    positions = positions[:-1]
    positions = positions.reshape(len(positions), -1)
    
    positions = np.concatenate([positions, velocity[:,:,0]], axis=-1)
    positions = np.concatenate([positions, velocity[:,:,2]], axis=-1)
    positions = np.concatenate([positions, rvelocity], axis=-1)
    
    """ Add Foot Contacts """
    positions = np.concatenate([positions, feet_l, feet_r], axis=-1)
        
    return positions

In [1671]:
Y_old = process_file2(files[3]);

In [1673]:
Y_old_hat = reconstruct_positions(Y_old, old_version=True)

In [ ]:
animation_plot(Y_old, fname="old", old_version=True)